In [ ]:
from google.colab import drive
drive.mount('/content/gdrive/')

In [ ]:
# MODIFY HERE TO YOUR PATH IN GOOGLE DRIVE
%cd /content/gdrive/MyDrive/CryptoML

In [ ]:
!pip install shap

In [ ]:
import speck as sp
import numpy as np
from tensorflow.keras.models import load_model

def create_testing_data(num_of_plaintexts, num_rounds):
    assert num_of_plaintexts in [2, 4]
    assert num_rounds > 0

    if num_of_plaintexts == 2:
        X, Y = sp.make_train_data_2pt(10**6, num_rounds)
        Xr, Yr = sp.real_differences_data_2pt(10**6, num_rounds)

    else:
        X, Y = sp.make_train_data_4pt(10**6, num_rounds)
        Xr, Yr = sp.real_differences_data_4pt(10**6, num_rounds)

    return X, Y, Xr, Yr

def get_inputs(num_of_plaintexts, num_rounds, num_of_samples, model_name):

    assert num_of_plaintexts in [2,4]
    assert num_rounds > 0
    assert num_of_samples > 1

    num_of_plaintexts = num_of_plaintexts
    num_rounds = num_rounds
    net = load_model('./models/' + model_name)
    num_of_samples = num_of_samples
    return num_of_plaintexts, num_rounds, num_of_samples, net


def evaluate(net,X,Y):
    Z = net.predict(X,batch_size=10000).flatten()
    Zbin = (Z > 0.5)
    diff = Y - Z 
    mse = np.mean(diff*diff)
    n = len(Z) 
    n0 = np.sum(Y==0) 
    n1 = np.sum(Y==1)
    acc = np.sum(Zbin == Y) / n
    tpr = np.sum(Zbin[Y==1]) / n1
    tnr = np.sum(Zbin[Y==0] == 0) / n0
    mreal = np.median(Z[Y==1])
    high_random = np.sum(Z[Y==0] > mreal) / n0
    print("Accuracy: ", acc, "TPR: ", tpr, "TNR: ", tnr, "MSE:", mse)
    print("Percentage of random pairs with score higher than median of real pairs:", 100 * high_random)

In [ ]:
import shap

# MODIFY HERE
num_of_plaintexts, num_rounds, num_of_samples, net = get_inputs(num_of_plaintexts=4, num_rounds=5, num_of_samples=10, model_name="best5depth10.h5")

X, Y, Xr, Yr = create_testing_data(num_of_plaintexts, num_rounds)

In [ ]:
print('Testing neural distinguishers against blocks in the ordinary real vs random setting')
evaluate(net, X, Y)

shap.initjs()
explainer = shap.KernelExplainer(net.predict, X[:num_of_samples])
shap_values = explainer.shap_values(X[:num_of_samples])
shap.force_plot(explainer.expected_value[0], shap_values[0], X)

In [ ]:
print('Testing real differences setting now.')
evaluate(net, Xr, Yr)

shap.initjs()
explainer = shap.KernelExplainer(net.predict, Xr[:num_of_samples])
shap_values = explainer.shap_values(Xr[:num_of_samples])
shap.force_plot(explainer.expected_value[0], shap_values[0], Xr)

In [ ]:
# TEST
import unittest

class TestEvalIPYNB(unittest.TestCase):
    
    def test_create_testing_data(self):
        # Invalid num_of_plaintexts
        with self.assertRaises(AssertionError):
            create_testing_data(num_of_plaintexts=0, num_rounds=5)
            
        with self.assertRaises(AssertionError):
            create_testing_data(num_of_plaintexts=10, num_rounds=5)
            
        # Invalid num_rounds
        with self.assertRaises(AssertionError):
            create_testing_data(num_of_plaintexts=2, num_rounds=0)

        # Valid num_of_plaintexts and num_rounds
        self.assertTrue(create_testing_data(num_of_plaintexts=2, num_rounds=1))
        self.assertTrue(create_testing_data(num_of_plaintexts=4, num_rounds=5))
        
    def test_get_inputs(self):
        
        dir = './models/'
        
        # Invalid num_of_plaintexts
        with self.assertRaises(AssertionError):
            get_inputs(num_of_plaintexts=1, num_rounds=5, num_of_samples=10, directory=dir+"best5depth10.h5")
            
        # Invalid num_rounds
        with self.assertRaises(AssertionError):
            get_inputs(num_of_plaintexts=2, num_rounds=0, num_of_samples=10, directory=dir+"best5depth10.h5")
            
        # Invalid num_of_samples
        with self.assertRaises(AssertionError):
            get_inputs(num_of_plaintexts=2, num_rounds=5, num_of_samples=1, directory=dir+"best5depth10.h5")
            
        # Invalid directory
        with self.assertRaises(OSError):
            get_inputs(num_of_plaintexts=4, num_rounds=5, num_of_samples=10, directory=dir+"doesnotexist.h5")
            
        self.assertTrue(get_inputs(num_of_plaintexts=4, num_rounds=5, num_of_samples=10, directory=dir+"best5depth10.h5"))
            
    def test_evaluate(self):
        
        X, Y, Xr, Yr = create_testing_data(4, 5)
        net = load_model(dir+"best5depth10.h5")
        
        # Invalid model
        with self.assertRaises(AttributeError):
            evaluate('notmodel', X, Y)
        
        # Invalid data X and Y
        with self.assertRaises(IndexError):
            evaluate(net, 1, 1)
                            
        X, Y, Xr, Yr = create_testing_data(2, 5)
        with self.assertRaises(ValueError):
            evaluate(net, X, Y)
        
if __name__ == '__main__': 
    unittest.main(argv=['first-arg-is-ignored'], exit=False, verbosity=2)